## Join

In [141]:
import pandas as pd
import sqlite3

### Connection to sqlite3

In [142]:
conn = sqlite3.connect("../ex00/data/checking-logs.sqlite")

### Created datamart

In [ ]:
query = """
SELECT
    c.uid,
    c.labname,
    MIN(c.timestamp) AS first_commit_ts,
    MIN(p.datetime) AS first_vie+w_ts
FROM checker c
LEFT JOIN pageviews p
    ON c.uid = p.uid
WHERE c.status = 'ready'
  AND c.numTrials = 1
  AND c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
  AND c.uid LIKE 'user_%'
GROUP BY c.uid, c.labname
"""
datamart = pd.io.sql.read_sql(query, conn, parse_dates=["first_commit_ts", "first_view_ts"])
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


### Test and control guruhga ajratish

In [144]:
test = datamart[datamart["first_view_ts"].notna()].copy()
test.head(10)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


In [145]:
control = datamart[datamart["first_view_ts"].isna()].copy()
control

,uid,labname,first_commit_ts,first_view_ts
12,user_11,laba05,2020-05-03 21:06:55.970293,NaT
13,user_11,project1,2020-05-03 23:45:33.673409,NaT
14,user_12,laba04,2020-04-18 17:07:51.767358,NaT
15,user_12,laba04s,2020-04-26 15:42:38.070593,NaT
16,user_12,laba05,2020-05-03 08:39:25.174316,NaT
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,NaT
136,user_8,laba05,2020-05-02 13:28:07.705193,NaT
137,user_8,laba06,2020-05-16 17:56:15.755553,NaT
138,user_8,laba06s,2020-05-16 20:01:07.900727,NaT


### controldagi bo‘sh first_view_ts qiymatlarini test dagi o‘rtacha vaqt bilan to‘ldirish

In [146]:
avg_view_ts = test["first_view_ts"].mean()
avg_view_ts

Timestamp('2020-04-27 00:40:05.761783552')

In [147]:
control["first_view_ts"] = avg_view_ts
control

,uid,labname,first_commit_ts,first_view_ts
12,user_11,laba05,2020-05-03 21:06:55.970293,2020-04-27 00:40:05.761783552
13,user_11,project1,2020-05-03 23:45:33.673409,2020-04-27 00:40:05.761783552
14,user_12,laba04,2020-04-18 17:07:51.767358,2020-04-27 00:40:05.761783552
15,user_12,laba04s,2020-04-26 15:42:38.070593,2020-04-27 00:40:05.761783552
16,user_12,laba05,2020-05-03 08:39:25.174316,2020-04-27 00:40:05.761783552
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,2020-04-27 00:40:05.761783552
136,user_8,laba05,2020-05-02 13:28:07.705193,2020-04-27 00:40:05.761783552
137,user_8,laba06,2020-05-16 17:56:15.755553,2020-04-27 00:40:05.761783552
138,user_8,laba06s,2020-05-16 20:01:07.900727,2020-04-27 00:40:05.761783552


### Written to sql

In [148]:
test.to_sql("test", conn, if_exists="replace", index=False)
control.to_sql("control", conn, if_exists="replace", index=False)

81

### Connection closed

In [149]:
conn.close()